# Stock Market Calendar  

### Past and Future dates when the stock market is open and closed

Produces 3 csv files, namely
1. All dates from 1927-12-30 to 2023-12-31 (all_dates.txt)
2. Dates when stock market is open (all_open_dates.txt)
3. Dates when stock market is closed (all_close_dates.txt)

In [1]:
import datetime
import os

import pandas as pd
from pandas_datareader._utils import RemoteDataError
import pandas_datareader.data as pdr
import yfinance as yf

# Override pandas_datareader with yfinance
yf.pdr_override()

In [2]:
start_date = datetime.date(1927 ,12, 30)
end_date   = datetime.date(2026, 12, 31)

### All dates (all_dates.txt)

In [3]:
filename = 'all_dates.txt'
all_dates = []

if not os.path.isfile(filename):
    all_dates = pd.date_range(start_date, end_date)
    all_dates = all_dates.strftime('%Y-%m-%d').to_list()
    with open (filename, 'w') as f:
        for date in all_dates:
            f.write(f'{date}\n')
with open (filename, 'r') as f:
    all_dates = [line.rstrip() for line in f]
all_dates[:10]

['1927-12-30',
 '1927-12-31',
 '1928-01-01',
 '1928-01-02',
 '1928-01-03',
 '1928-01-04',
 '1928-01-05',
 '1928-01-06',
 '1928-01-07',
 '1928-01-08']

### Past Open Dates

First we need to get the trading dates from the past.  We'll get the entire timeseries for S&P 500 and assume that its dates are accuracte as to when trading was allowed.

In [7]:
symbol = '^GSPC'
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()
timeseries_cache = symbol + '.csv'

if os.path.isfile(timeseries_cache):
    pass
else:
    try:
        ts = pdr.get_data_yahoo(symbol, start=datetime.datetime(1900, 1, 1), progress=False)
    except RemoteDataError as e:
        print(f'\n{e}')
    except Exception as e:
        print(f'\n{e}')
    else:
        ts.to_csv(timeseries_cache, encoding='utf-8')

ts = pd.read_csv(timeseries_cache, index_col='Date', parse_dates=True)
ts

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1927-12-30,17.660000,17.660000,17.660000,17.660000,17.660000,0
1928-01-03,17.760000,17.760000,17.760000,17.760000,17.760000,0
1928-01-04,17.719999,17.719999,17.719999,17.719999,17.719999,0
1928-01-05,17.549999,17.549999,17.549999,17.549999,17.549999,0
1928-01-06,17.660000,17.660000,17.660000,17.660000,17.660000,0
...,...,...,...,...,...,...
2023-12-21,4724.290039,4748.709961,4708.350098,4746.750000,4746.750000,3431180000
2023-12-22,4753.919922,4772.939941,4736.770020,4754.629883,4754.629883,3046770000
2023-12-26,4758.859863,4784.720215,4758.450195,4774.750000,4774.750000,2513910000


Get the dates ctock market was open in the past.

In [8]:
# Drop all the columns
ts = ts.drop(ts.columns, axis=1)
past_open_dates = ts.index.strftime('%Y-%m-%d').to_list()
past_open_dates[:10]

['1927-12-30',
 '1928-01-03',
 '1928-01-04',
 '1928-01-05',
 '1928-01-06',
 '1928-01-09',
 '1928-01-10',
 '1928-01-11',
 '1928-01-12',
 '1928-01-13']

### Future Open Dates

We will use a list of the US holiday schedule for 2021-2023.  We take those days away from complete business calander (Mon through Fri) for 2021-2023.

List of dates market is closed in 2021-2023 for US holidays:¶

https://www.nyse.com/markets/hours-calendars


In [9]:
future_holidays = [
    # 2024
    '2024-01-01', '2024-01-15', '2024-02-19', '2024-03-29', '2024-05-27',
    '2024-06-19', '2024-07-04', '2024-09-02', '2024-11-28', '2024-12-25',
    # 2025
    '2025-01-01', '2025-01-20', '2025-02-17', '2025-04-18', '2025-05-26',
    '2024-06-19', '2025-07-04', '2025-09-01', '2025-11-27', '2025-12-25',
    # 2025
    '2025-01-01', '2025-01-19', '2025-02-16', '2025-04-03', '2025-05-25',
    '2024-06-19', '2025-07-03', '2025-09-07', '2025-11-26', '2025-12-25'
]

Get list of future business dates.

In [10]:
start = datetime.date(2024 ,1, 1)
end   = datetime.date(2026, 12, 31)

future_business_dates = pd.date_range(start, end, freq='B')
future_business_dates = future_business_dates.strftime('%Y-%m-%d').to_list()
future_business_dates[:10]

['2024-01-01',
 '2024-01-02',
 '2024-01-03',
 '2024-01-04',
 '2024-01-05',
 '2024-01-08',
 '2024-01-09',
 '2024-01-10',
 '2024-01-11',
 '2024-01-12']

Remove the holidays from the business dates and that will be the future market open dates.

In [11]:
future_open_dates = list(set(future_business_dates) - set(future_holidays))
future_open_dates = sorted(future_open_dates)
future_open_dates[:10]

['2024-01-02',
 '2024-01-03',
 '2024-01-04',
 '2024-01-05',
 '2024-01-08',
 '2024-01-09',
 '2024-01-10',
 '2024-01-11',
 '2024-01-12',
 '2024-01-16']

### All Open Dates

Finally, join the past and future market open lists.

In [12]:
all_open_dates = set(past_open_dates).union(set(future_open_dates))
all_open_dates = sorted(all_open_dates)
all_open_dates[:10]

['1927-12-30',
 '1928-01-03',
 '1928-01-04',
 '1928-01-05',
 '1928-01-06',
 '1928-01-09',
 '1928-01-10',
 '1928-01-11',
 '1928-01-12',
 '1928-01-13']

In [13]:
filename = 'all_open_dates.txt'
with open (filename, 'w') as f:
    for date in all_open_dates:
        f.write(f'{date}\n')

In [14]:
filename = 'stock_market_calendar.py'
with open (filename, 'w') as f:
    f.write('"""\n')
    f.write('Past and Future dates when the stock market is open from 1928 to 2024.\n')
    f.write('"""\n\n')
    f.write(f'# Auto-generated with stock_market_calendar.ipynb, see:\n')
    f.write(f'# https://github.com/fja05680/stock_market_calendar\n\n')
    f.write(f'stock_market_calendar = [\n')
    for date in all_open_dates[:-1]:
        f.write(f"    '{date}',\n")
    f.write(f"    '{date}'\n]")

### All Close Dates

In [15]:
all_close_dates = list(set(all_dates) - set(all_open_dates))
all_close_dates = sorted(all_close_dates)
all_close_dates[:10]

['1927-12-31',
 '1928-01-01',
 '1928-01-02',
 '1928-01-07',
 '1928-01-08',
 '1928-01-14',
 '1928-01-15',
 '1928-01-21',
 '1928-01-22',
 '1928-01-28']

In [16]:
filename = 'all_close_dates.txt'
with open (filename, 'w') as f:
    for date in all_close_dates:
        f.write(f'{date}\n')